In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
⠹ Installing integrations...


{'status': 'ok', 'restart': True}

In [ ]:
NGROK_TOKEN = "2faRklfq8UyCoy9ZEuNKUekXYJl_6z233LZERHvZ1M4uiMda2"

In [ ]:
from zenml.environment import Environment

In [ ]:
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.56.4
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
⠼ Initializing ZenML repository at /content.
Creating default stack in workspace default...
⠴ Initializing ZenML repository at /content.
Creating default user 'default' ...
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  """Train and test a scikit-learn SVC classifier on digits"""
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc = model.score(X_test,y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9583333333333334


In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"]
]:
    """Load the digits datasets as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray
) -> ClassifierMixin:
    """Train an sklearn SVC classifier."""
    model = SVC(gamma=0.001)
    model.fit(X_train,y_train)
    return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc

In [ ]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    """Links all the steps together in the pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train,y_train=y_train)
    evaluator(X_test=X_test,y_test=y_test,model=model)

In [ ]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Step svc_trainer has started.
Step svc_trainer has finished in 0.474s.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 0.676s.
Pipeline run has finished in 1.576s.


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")

    else:
        !zenml up --port {port}

start_zenml_dashboard()


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-f0ec60e0-af3e-4d83-8710-e917e183f73c


Opening tunnel named: http-8237-f0ec60e0-af3e-4d83-8710-e917e183f73c


INFO:pyngrok.process.ngrok:t=2024-04-25T10:16:40+0000 lvl=info msg="no configuration paths supplied"


t=2024-04-25T10:16:40+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-04-25T10:16:40+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-04-25T10:16:40+0000 lvl=info msg=start pg=/api/tunnels id=7bbd0911efdff792


INFO:pyngrok.process.ngrok:t=2024-04-25T10:16:40+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-f0ec60e0-af3e-4d83-8710-e917e183f73c addr=http://localhost:8237 url=https://cbe8-34-138-170-179.ngrok-free.app


In Colab, use this URL instead: NgrokTunnel: "https://cbe8-34-138-170-179.ngrok-free.app" -> "http://localhost:8237"!
t=2024-04-25T10:16:40+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-f0ec60e0-af3e-4d83-8710-e917e183f73c addr=http://localhost:8237 url=https://cbe8-34-138-170-179.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-04-25T10:16:40+0000 lvl=info msg=end pg=/api/tunnels id=7bbd0911efdff792 status=201 dur=46.017669ms


t=2024-04-25T10:16:40+0000 lvl=info msg=end pg=/api/tunnels id=7bbd0911efdff792 status=201 dur=46.017669ms
